In [1]:
# We have considered the wine quality dataset to implement bagging. 
# Here with the bagging we are classifying the Wine as Red or White. 
# The dataset has chemical indicators and quality metrics with which we can determine the class of the wine. 
from __future__ import print_function
import os
data_path = ['data']

from sklearnex import patch_sklearn
patch_sklearn()
#unpatch_sklearn()

from io import StringIO
from IPython.display import Image, display
import time

from sklearn.tree import export_graphviz
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [5]:
import pandas as pd
import numpy as np
# loading data
filepath = 'Wine_Quality_Data.csv'
data = pd.read_csv(filepath, sep=',')
data['color'] = data.color.replace('white',0).replace('red',1).astype(int)
# All data columns except for color
feature_cols = [x for x in data.columns if x not in 'color']
# Split the data into two parts with 1000 points in the test data
# This creates a generator
strat_shuff_split = StratifiedShuffleSplit(n_splits=1, test_size=1000, random_state=42)

# Get the index values from the generator
train_idx, test_idx = next(strat_shuff_split.split(data[feature_cols], data['color']))

# Create the data sets
X_train = data.loc[train_idx, feature_cols]
y_train = data.loc[train_idx, 'color']

X_test = data.loc[test_idx, feature_cols]
y_test = data.loc[test_idx, 'color']

strat_shuff_split.get_n_splits(X_train,y_test)

1

In [11]:
from sklearnex import patch_sklearn, unpatch_sklearn
unpatch_sklearn

from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest estimator
# Note that the number of trees is not setup here
RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)
#out of bag score list
oob_list = list()

# Iterate through all of the possibilities for 
# number of trees

for n_trees in [25, 50, 100, 150, 200, 300, 400]:
    
    # Use this to set the number of trees
    RF.set_params(n_estimators=n_trees)
    
    # Time fit function
    t = time.process_time()

    # Fit the model
    RF.fit(X_train, y_train)
    elapsed_time = time.process_time() - t
    
    # Get the oob error
    oob_error = 1 - RF.oob_score_
    
    # Store it
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

print ("It took",elapsed_time," to fit.")

/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `m

It took 2.069033829999995  to fit.


In [12]:
from sklearnex import patch_sklearn, unpatch_sklearn
#unpatch_sklearn
patch_sklearn()
from sklearn.ensemble import RandomForestClassifier

# Initialize the random forest estimator
# Note that the number of trees is not setup here
RF = RandomForestClassifier(oob_score=True, 
                            random_state=42, 
                            warm_start=True,
                            n_jobs=-1)
#out of bag score list
oob_list = list()

# Iterate through all of the possibilities for 
# number of trees
for n_trees in [25, 50, 100, 150, 200, 300, 400]:
    
    # Use this to set the number of trees
    RF.set_params(n_estimators=n_trees)
    
    # Time fit function
    t = time.process_time()

    # Fit the model
    RF.fit(X_train, y_train)
    elapsed_time = time.process_time() - t
    
    # Get the oob error
    oob_error = 1 - RF.oob_score_
    
    # Store it
    oob_list.append(pd.Series({'n_trees': n_trees, 'oob': oob_error}))

rf_oob_df = pd.concat(oob_list, axis=1).T.set_index('n_trees')

print ("It took",elapsed_time," to fit.")

rf_oob_df

# We are iterating through number of trees for choosing the hyper parameters and we check 25 trees, 50 trees and 
# so on till 400 trees. We found that 150 trees is having the least error and we will be using that as the 
# Hyper parameter to do the fit. The time taken for the model to fit is 1.7022535219999995 which is appreciable. 
# Again the stock version this is a considerable improvement with 36% reduction in the execution time. 

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has bee

It took 1.7022535219999995  to fit.


,oob
n_trees,
25.0,0.008004
50.0,0.005821
100.0,0.005094
150.0,0.004730
200.0,0.005094
300.0,0.004912
400.0,0.005094


In [13]:
100*(2.069033829999995 - 1.7022535219999995)
# one could see that we have 36% improvement in the performance with the intel optimisation. 

36.67803079999956